In [11]:
# Importazione delle librerie necessarie
import os
import numpy as np
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
import librosa.display
from pydub import AudioSegment
from tqdm import tqdm  # Importa la libreria tqdm per la progress bar

In [12]:
# Definizione delle directory di input e output
input_dir = 'dataset_5'
output_dir = 'NuovoDataset'

# Creare la directory di output se non esiste
os.makedirs(output_dir, exist_ok=True)

# Definizione della lunghezza desiderata in secondi e in campioni
desired_length_sec = 4  # Durata desiderata in secondi
sampling_rate = 192000  # Frequenza di campionamento (modifica se necessario)
desired_length_samples = int(desired_length_sec * sampling_rate)

In [13]:
# Lista per memorizzare i percorsi dei file audio
audio_files = []

# Scorrere ricorsivamente le cartelle nel dataset
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.wav') or file.endswith('.mp3'):
            audio_files.append(os.path.join(root, file))

# Visualizzare il numero di file audio trovati
print(f"Numero di file audio trovati: {len(audio_files)}")

Numero di file audio trovati: 5


In [14]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os

def preprocess_audio(file, input_dir, output_dir, segment_length_seconds, sampling_rate):
    # Carica l'audio utilizzando librosa (sia MP3 che WAV)
    y, sr = librosa.load(file, sr=sampling_rate, mono=True)
    
    # Calcola la lunghezza di ciascun segmento in campioni
    segment_length_samples = segment_length_seconds * sampling_rate
    
    # Calcola il numero di segmenti
    num_segments = int(np.ceil(len(y) / segment_length_samples))
    
    # Creare la struttura delle directory di output mantenendo la stessa gerarchia
    relative_path = os.path.relpath(file, input_dir)
    base_output_path = os.path.splitext(os.path.join(output_dir, relative_path))[0]
    
    for i in range(num_segments):
        start_sample = i * segment_length_samples
        end_sample = min(start_sample + segment_length_samples, len(y))
        y_segment = y[start_sample:end_sample]
        
        # Assicura che la lunghezza del segmento sia corretta
        if len(y_segment) < segment_length_samples:
            y_segment = np.pad(y_segment, (0, segment_length_samples - len(y_segment)), mode='constant')
        
        # Crea lo spettrogramma utilizzando la STFT
        D = np.abs(librosa.stft(y_segment))  # Magnitudine della STFT
        D_db = librosa.amplitude_to_db(D, ref=np.max)
        
        # Salva lo spettrogramma come immagine
        spectrogram_output_file = f"{base_output_path}_segment_{i+1}.png"
        os.makedirs(os.path.dirname(spectrogram_output_file), exist_ok=True)
        
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(D_db, sr=sr, x_axis='time', y_axis='log')
        plt.colorbar(format='%+2.0f dB')
        plt.title(f'Spectrogram (STFT) - Segment {i+1}')
        plt.tight_layout()
        plt.savefig(spectrogram_output_file, bbox_inches='tight', pad_inches=0.1)
        plt.close()

# Esempio di utilizzo dello script
input_dir = 'dataset_5'
output_dir = 'NuovoDataset'
segment_length_seconds = 4  # Lunghezza di ciascun segmento in secondi
sampling_rate = 22050  # o qualsiasi altra frequenza di campionamento desiderata

# Itera attraverso i file nell'input directory e genera spettrogrammi
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.mp3') or file.endswith('.wav'):
            file_path = os.path.join(root, file)
            preprocess_audio(file_path, input_dir, output_dir, segment_length_seconds, sampling_rate)


In [15]:
# Preprocessamento di ciascun file audio con progress bar
with tqdm(total=len(audio_files), desc='Processing audio files') as progress_bar:
    for file in audio_files:
        preprocess_audio(file, output_dir, desired_length_samples, sampling_rate)
        progress_bar.update(1)

print("Tutti gli spettrogrammi sono stati generati e salvati come immagini.")

Processing audio files:   0%|          | 0/5 [00:00<?, ?it/s]


TypeError: preprocess_audio() missing 1 required positional argument: 'sampling_rate'